In [1]:
from submodules import Objects, Robots, Tasks, Users
from submodules.Scenes import Scene
from submodules.Actions import Actions
from submodules.Gestures import Gestures
from submodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a

# Generating data

It is possible to add the seventh Action: `put_on_target` which puts the holding/attached object on random place in the scene.

In [2]:
Actions.A

['move_up', 'open', 'put', 'pour', 'close', 'pick_up']

In [3]:
Actions.A.append('put_on_target')

In [4]:
Actions.A

['move_up', 'open', 'put', 'pour', 'close', 'pick_up', 'put_on_target']

Function `regulize_repeating_pattern` checks if the current action draw is not the same as the last two draws. 

In [5]:
def regulize_repeating_pattern(TaTos, TaTo):
    if len(TaTos) < 2: return True
    
    if TaTo == TaTos[-1] or TaTo == TaTos[-2]:
        return False
    return True

## Ex.1: Generate random draws from all actions

Generate random valid action draws from the input created scene.

Each action has `common_sense_probability` threshold `p>0.9` which rejects the action that has probability under this value.
If any action is not found over `common_sense_probability` threshold. The next draws are chosen based on if the action is feasible/doable `p>0.01`.

In [6]:
draws = 20
common_sense_probability_threshold = 0.9
feasibility_probability_threshold = 0.01

In [7]:
# Generate random scene
s = Scene(init='drawer,cup,cup')

TaTos = []
# Generate random action
for i in range(draws):
    n=0
    while True:
        # random draw from all actions
        TaTo = (Actions.get_random_action(), s.get_random_object())
        # regulizer
        while not regulize_repeating_pattern(TaTos, TaTo):
            TaTo = (Actions.get_random_action(), s.get_random_object())
        
        success = Actions.do(s, TaTo, ignore_location=True, p=common_sense_probability_threshold)
        if n>100:
            success = Actions.do(s, TaTo, ignore_location=True, p=feasibility_probability_threshold)
        if success:
            TaTos.append(TaTo)
            break
        n+=1

In [8]:
TaTos

[('pick_up', 'cup1'),
 ('put_on_target', 'drawer'),
 ('open', 'drawer'),
 ('pick_up', 'cup'),
 ('put_on_target', 'cup1'),
 ('close', 'drawer'),
 ('pick_up', 'cup1'),
 ('put_on_target', 'drawer'),
 ('pick_up', 'cup'),
 ('put_on_target', 'cup1'),
 ('pick_up', 'cup1'),
 ('put_on_target', 'drawer'),
 ('open', 'drawer'),
 ('close', 'drawer'),
 ('pick_up', 'cup'),
 ('open', 'drawer'),
 ('put_on_target', 'drawer'),
 ('pick_up', 'cup1'),
 ('close', 'drawer'),
 ('put_on_target', 'cup1')]

In [9]:
s.info

Scene info. shape: [4, 4, 4]
drawer, drawer, [1 3 0], closed, []
cup, cup, [0 2 0], empty
cup1, cup, [3 3 0], empty
Robot: [3 1 3], opened, rotation: 0, attached: -


TaTos variable contains possible actions which can be executed from newly created scene

In [10]:
s = Scene(init='drawer,cup,cup')
for TaTo in TaTos:
    Actions.do(s, TaTo)

In [11]:
s.info

Scene info. shape: [4, 4, 4]
drawer, drawer, [1 0 0], closed, []
cup, cup, [3 1 0], empty
cup1, cup, [1 1 0], empty
Robot: [1 0 3], opened, rotation: 0, attached: -


## Ex. 2: Generate random draws from possible actions
- TODO: Maybe to 

In [15]:
# Generate random scene
s = Scene(init='drawer,cup,cup')

TaTos = []
# Generate random action
for i in range(draws):
    n=0
    while True:
        # Random draw from possible actions
        TaTo = Actions.get_random_possible_action(s, ignore_location=True, p=common_sense_probability_threshold)
        flag_feasible_option = False
        while not regulize_repeating_pattern(TaTos, TaTo):
            flag_feasible_option = False
            TaTo = Actions.get_random_possible_action(s, ignore_location=True, p=common_sense_probability_threshold)
            if TaTo is False:
                flag_feasible_option = True
                TaTo = Actions.get_random_possible_action(s, ignore_location=True, p=feasibility_probability_threshold)
                
        # Action execution
        if flag_feasible_option:
            success = Actions.do(s, TaTo, ignore_location=True, p=feasibility_probability_threshold)
        else:
            success = Actions.do(s, TaTo, ignore_location=True, p=common_sense_probability_threshold)
        if success:
            TaTos.append(TaTo)
            break


In [32]:
TaTos

[('close', 'drawer'),
 ('open', 'drawer'),
 ('close', 'drawer'),
 ('open', 'drawer'),
 ('close', 'drawer'),
 ('open', 'drawer'),
 ('close', 'drawer'),
 ('open', 'drawer'),
 ('close', 'drawer'),
 ('open', 'drawer')]